In [9]:
import findspark
findspark.init("/opt/spark")

from pyspark.sql import SparkSession 

In [10]:
spark = (
    SparkSession
        .builder
        .master('local[4]') 
        .appName('mi_app')
        .getOrCreate()
)

### Load Data

In [13]:
df = spark.read.format('json').load('/home/gonzalo/Documents/notes/notes-spark/data/flight-data/json/2015-summary.json')

In [14]:
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



In [16]:
df.schema

StructType([StructField('DEST_COUNTRY_NAME', StringType(), True), StructField('ORIGIN_COUNTRY_NAME', StringType(), True), StructField('count', LongType(), True)])

In [15]:
df.show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|   15|
|       United States|            Croatia|    1|
|       United States|            Ireland|  344|
|               Egypt|      United States|   15|
|       United States|              India|   62|
|       United States|          Singapore|    1|
|       United States|            Grenada|   62|
|          Costa Rica|      United States|  588|
|             Senegal|      United States|   40|
|             Moldova|      United States|    1|
|       United States|       Sint Maarten|  325|
|       United States|   Marshall Islands|   39|
|              Guyana|      United States|   64|
|               Malta|      United States|    1|
|            Anguilla|      United States|   41|
|             Bolivia|      United States|   30|
|       United States|           Paraguay|    6|
|             Algeri

In [18]:
# Los schemas pueden ser definidos manualmente
# antes de importar data

from pyspark.sql.types import StructField, StructType, StringType, LongType

my_schema = StructType([
    StructField('DEST_COUNTRY_NAME', StringType(), True),
    StructField('ORIGIN_COUNTRY_NAME', StringType(), True),
    StructField('count', LongType(), False, metadata={'hello': 'world'})
])

df = (
    spark.read
    .format('json')
    .schema(my_schema)
    .load('/home/gonzalo/Documents/notes/notes-spark/data/flight-data/json/2015-summary.json')
)

In [20]:
df.schema

StructType([StructField('DEST_COUNTRY_NAME', StringType(), True), StructField('ORIGIN_COUNTRY_NAME', StringType(), True), StructField('count', LongType(), True)])

In [21]:
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



### Creacion DF

In [22]:
from pyspark import Row
from pyspark.sql.types import StructField, StructType, StringType, LongType

my_sch = StructType(
    [
        StructField('name', StringType(), True),
        StructField('age', LongType(), True),
        StructField('city', StringType(), True)
    ]
)

# Es posible definir una DF a partir de ROWS y un SCHEMA
myRow = Row('Gonzalo', 25, 'Cordoba')

df = spark.createDataFrame([myRow], my_sch)

df.show()

+-------+---+-------+
|   name|age|   city|
+-------+---+-------+
|Gonzalo| 25|Cordoba|
+-------+---+-------+



### VIEWS

In [23]:
from pyspark.sql.types import StructField, StructType, StringType, LongType

my_schema = StructType([
    StructField('DEST_COUNTRY_NAME', StringType(), True),
    StructField('ORIGIN_COUNTRY_NAME', StringType(), True),
    StructField('count', LongType(), False, metadata={'hello': 'world'})
])

df = (
    spark.read
    .format('json')
    .schema(my_schema)
    .load('/home/gonzalo/Documents/notes/notes-spark/data/flight-data/json/2015-summary.json')
)

In [24]:
# Genero una view temporal posible de ser consultada por SQL
flightData2015.createOrReplaceTempView("flight_data_2015")

In [26]:
sqlWay = spark.sql("""
select dest_country_name, count(1)
from flight_data_2015
group by dest_country_name
""")

sqlWay.show()

+--------------------+--------+
|   dest_country_name|count(1)|
+--------------------+--------+
|            Anguilla|       1|
|              Russia|       1|
|            Paraguay|       1|
|             Senegal|       1|
|              Sweden|       1|
|            Kiribati|       1|
|              Guyana|       1|
|         Philippines|       1|
|            Djibouti|       1|
|            Malaysia|       1|
|           Singapore|       1|
|                Fiji|       1|
|              Turkey|       1|
|                Iraq|       1|
|             Germany|       1|
|              Jordan|       1|
|               Palau|       1|
|Turks and Caicos ...|       1|
|              France|       1|
|              Greece|       1|
+--------------------+--------+
only showing top 20 rows

